# Algorytm wspinaczki z multistartem (iteracyjna wspinaczka - IHC) 

Grupa: Amelia Madej, Justyna Sarkowicz, Olga Sieradzan, Weronika Duda i Aleksandra Węgrzyn

## Biblioteki

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

## Pobór danych

In [ ]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaj ruchów

Poniżej znajduje się implementacja trzech rodzajów ruchów (generowania rozwiązań sąsiednich)

In [3]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Długość ścieżki

In [4]:
def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Implementacja algorytmu

In [6]:
# Mapowanie typów sąsiedztwa na funkcje
neighborhood_functions = {
    "swap": swap_move,
    "two_opt": two_opt_move,
    "insertion": insertion_move
}

def hill_climbing(distance_matrix, initial_route, neighborhood_type, max_iterations_without_improvement, max_hill_climbing_iterations):
    current_route = initial_route
    current_distance = calculate_path_cost(distance_matrix, current_route)
    
    # Wybierz funkcję sąsiedztwa na podstawie typu
    neighbor_func = neighborhood_functions[neighborhood_type]
    iterations_without_improvement = 0
    total_iterations = 0

    while iterations_without_improvement < max_iterations_without_improvement and total_iterations < max_hill_climbing_iterations:
        improved = False
        # Przeglądaj sąsiedztwa
        for neighbor in neighbor_func(current_route):
            new_distance = calculate_path_cost(distance_matrix, neighbor)
            if new_distance < current_distance:
                current_route = neighbor
                current_distance = new_distance
                improved = True
                iterations_without_improvement = 0
                break  # Skok do następnego sąsiedztwa

        if not improved:
            # Jeśli nie znaleziono lepszego rozwiązania, zwiększ licznik iteracji bez poprawy
            iterations_without_improvement += 1
        
        # Zwiększ licznik iteracji
        total_iterations += 1

    return current_route, current_distance

def iterated_hill_climbing(distance_matrix, num_starts=100, neighborhood_type="two_opt", time_limit=500, max_iterations_without_improvement=100, max_hill_climbing_iterations=1000):
    start_time = time.time()  # Czas rozpoczęcia

    n = len(distance_matrix)
    best_route = None
    best_distance = float('inf')
    best_time = float('inf')

    for start in range(num_starts):
        # Sprawdzenie limitu czasu
        if time_limit and (time.time() - start_time > time_limit):
            print(f"Przekroczono limit czasu ({time_limit} s).")
            break

        # Losowe rozwiązanie początkowe
        initial_route = list(range(n))
        random.shuffle(initial_route)

        # Wspinaczka
        hill_climbing_start_time = time.time()
        current_route, current_distance = hill_climbing(distance_matrix, initial_route, neighborhood_type, max_iterations_without_improvement, max_hill_climbing_iterations)
        hill_climbing_time = time.time() - hill_climbing_start_time

        # Aktualizacja najlepszego rozwiązania
        if current_distance < best_distance:
            best_route = current_route
            best_distance = current_distance
            best_time = hill_climbing_time

    best_route = [x + 1 for x in best_route]  # Zmień indeksy na numery miast
    return best_route, best_distance, best_time

## Generowanie wyników

Poniżej znajduje się zestawienie wyników z uwzględnieniem wpływu na wyniki różnych wartości parametrów algorytmu

Za podstawowe dane przyjmujemy:

* num_starts = 100

* neighborhood_type = `two_opt`

* time_limit = 500

* max_iterations_without_improvement = 100

* max_hill_climbing_iterations = 1000

Następnie badane są wpływy zmian wartości poszczególnych parametrów 

In [11]:
def run_multiple_tests(data, dataset_name, num_tests=10):
    results = []
    for _ in range(num_tests):
        best_path, best_cost, best_time = iterated_hill_climbing(data)
        results.append({
            "WYNIK": best_cost,
            "ŚCIEŻKA": best_path,
            "CZAS": best_time,
            "DATASET": dataset_name
        })
    return results

In [12]:
all_results = []
all_results.extend(run_multiple_tests(data1, "DATA1"))
all_results.extend(run_multiple_tests(data2, "DATA2"))
all_results.extend(run_multiple_tests(data3, "DATA3"))

basic = pd.DataFrame(data = all_results)

with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='w') as writer:
    basic.to_excel(writer, sheet_name = "Basic", index=False)

Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).


### Badanie wpływu zmian wartości parametru `num_starts`

In [10]:
def test_num_starts(data, dataset_name, num_repeats=10):
    results = []
    num_starts_values = [50, 75, 100, 125, 150]
    for num_starts in num_starts_values:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, num_starts=num_starts)
            results.append({"PARAMETR": num_starts, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_num_starts_1 = test_num_starts(data1, "DATA1")
df_num_starts_2 = test_num_starts(data2, "DATA2")
df_num_starts_3 = test_num_starts(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
num_starts = pd.concat([df_num_starts_1, df_num_starts_2, df_num_starts_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    num_starts.to_excel(writer, sheet_name = "num_starts", index=False)

Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit c

### Badanie wpływu zmian wartości parametru `neighborhood_type`

In [11]:
def test_neighborhood_type(data, dataset_name, num_repeats=10):
    results = []
    neighborhood_types = ["swap", "two_opt", "insertion"]
    for neighborhood_type in neighborhood_types:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, neighborhood_type=neighborhood_type)
            results.append({"PARAMETR": neighborhood_type, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_neighborhood_type_1 = test_neighborhood_type(data1, "DATA1")
df_neighborhood_type_2 = test_neighborhood_type(data2, "DATA2")
df_neighborhood_type_3 = test_neighborhood_type(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
neighborhood_type_results = pd.concat([df_neighborhood_type_1, df_neighborhood_type_2, df_neighborhood_type_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    neighborhood_type_results.to_excel(writer, sheet_name="neighborhood_type", index=False)

Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).
Przekroczono limit c

### Badanie wpływu zmian wartości parametru `time_limit`

In [ ]:
def test_time_limit(data, dataset_name, num_repeats=10):
    results = []
    time_limits = [200, 600, 1000, 1400]
    for time_limit in time_limits:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, time_limit=time_limit)
            results.append({"PARAMETR": time_limit, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_time_limit_1 = test_time_limit(data1, "DATA1")
df_time_limit_2 = test_time_limit(data2, "DATA2")
df_time_limit_3 = test_time_limit(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
time_limit_results = pd.concat([df_time_limit_1, df_time_limit_2, df_time_limit_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    time_limit_results.to_excel(writer, sheet_name="time_limit", index=False)

### Badanie wpływu zmian wartości parametru `max_iterations_without_improvement`

In [ ]:
def test_max_iterations_without_improvement(data, dataset_name, num_repeats=10):
    results = []
    max_iterations_values = [50, 75, 100, 125, 150]
    for max_iterations in max_iterations_values:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, max_iterations_without_improvement=max_iterations)
            results.append({"PARAMETR": max_iterations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_max_iterations_1 = test_max_iterations_without_improvement(data1, "DATA1")
df_max_iterations_2 = test_max_iterations_without_improvement(data2, "DATA2")
df_max_iterations_3 = test_max_iterations_without_improvement(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
max_iterations_results = pd.concat([df_max_iterations_1, df_max_iterations_2, df_max_iterations_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    max_iterations_results.to_excel(writer, sheet_name="max_iterations_without_improvement", index=False)

### Badanie wpływu zmian wartości parametru `max_hill_climbing_iterations`

In [ ]:
def test_max_hill_climbing_iterations(data, dataset_name, num_repeats=10):
    results = []
    max_hill_climbing_iterations_values = [250, 500, 750, 1000]
    for max_iterations in max_hill_climbing_iterations_values:
        for _ in range(num_repeats):
            best_path, best_cost, best_time = iterated_hill_climbing(data, max_hill_climbing_iterations=max_iterations)
            results.append({"PARAMETR": max_iterations, "WYNIK_1": best_cost, "Ścieżka": best_path, "CZAS": best_time})
        print(f"Gotowe {max_iterations} dla {dataset_name}")
    df_results = pd.DataFrame(results)
    df_results["DATASET"] = dataset_name
    return df_results

# Dla 3 zestawów danych
df_max_hill_climbing_1 = test_max_hill_climbing_iterations(data1, "DATA1")
df_max_hill_climbing_2 = test_max_hill_climbing_iterations(data2, "DATA2")
df_max_hill_climbing_3 = test_max_hill_climbing_iterations(data3, "DATA3")

# Łączenie wyników w jeden DataFrame
max_hill_climbing_results = pd.concat([df_max_hill_climbing_1, df_max_hill_climbing_2, df_max_hill_climbing_3], ignore_index=True)
with pd.ExcelWriter('IHC.xlsx', engine='openpyxl', mode='a') as writer:
    max_hill_climbing_results.to_excel(writer, sheet_name="max_hill_climbing_iterations", index=False)

Przekroczono limit czasu (500 s).
Gotowe 250 dla DATA1
Gotowe 500 dla DATA1
Przekroczono limit czasu (500 s).
Gotowe 750 dla DATA1
Przekroczono limit czasu (500 s).
Gotowe 1000 dla DATA1
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Gotowe 250 dla DATA2
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
Gotowe 500 dla DATA2
Przekroczono limit czasu (500 s).
Przekroczono limit czasu (500 s).
